In [75]:
import pandas as pd
from sklearn.metrics import confusion_matrix, make_scorer, fbeta_score
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
import warnings 
import sort_dataframeby_monthorweek as sd
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# warnings.filterwarnings("ignore")

# data_df = pd.read_csv("hotel_bookings.csv")
# pd.set_option("display.float_format", lambda x: "%.5f" % x)
# # pd.options.display.max_rows = 200000
# data_df = data_df.drop(columns=["reservation_status_date", "hotel", "reservation_status",
#     "arrival_date_week_number", "reserved_room_type", "company"])

# filter = (data_df.children == 0) & (data_df.adults == 0) & (data_df.babies == 0)
# data_df[filter]
# data_df = data_df[~filter]
# data_df=data_df.replace(" ","_")
# data_df = data_df.reindex(data_df.columns.tolist(), axis = 1)

# nan_replacements = {"children": 0.0, "agent": 0, "country": "unknown"}
# data_df = data_df.fillna(nan_replacements)

# data_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# # Drop rows with NaN
# data_df.dropna(inplace=True)

# data_df.drop(14969)

# features = data_df.drop(columns=["is_canceled"], axis=1).columns
# xs = data_df[features]
# ys = data_df["is_canceled"]
# le1 = LabelEncoder()
# le2 = LabelEncoder()
# le3 = LabelEncoder()
# le4 = LabelEncoder()
# le5 = LabelEncoder()
# le6 = LabelEncoder()
# le7 = LabelEncoder()
# le8 = LabelEncoder()

# le1.fit(xs['arrival_date_month'])
# tmp_s = le1.transform(xs['arrival_date_month']).reshape((-1,1))
# xs['arrival_date_month'] = pd.DataFrame(tmp_s)

# le2.fit(xs['meal'])
# tmp_s = le2.transform(xs['meal']).reshape((-1,1))
# xs['meal'] = pd.DataFrame(tmp_s)

# le3.fit(xs['market_segment'])
# tmp_s = le3.transform(xs['market_segment']).reshape((-1,1))
# xs['market_segment'] = pd.DataFrame(tmp_s)

# le4.fit(xs['country'])
# tmp_s = le4.transform(xs['country']).reshape((-1,1))
# xs['country'] = pd.DataFrame(tmp_s)

# le5.fit(xs['distribution_channel'])
# tmp_s = le5.transform(xs['distribution_channel']).reshape((-1,1))
# xs['distribution_channel'] = pd.DataFrame(tmp_s)

# le6.fit(xs['assigned_room_type'])
# tmp_s = le6.transform(xs['assigned_room_type']).reshape((-1,1))
# xs['assigned_room_type'] = pd.DataFrame(tmp_s)

# le7.fit(xs['deposit_type'])
# tmp_s = le7.transform(xs['deposit_type']).reshape((-1,1))
# xs['deposit_type'] = pd.DataFrame(tmp_s)

# le8.fit(xs['customer_type'])
# tmp_s = le8.transform(xs['customer_type']).reshape((-1,1))
# xs['customer_type'] = pd.DataFrame(tmp_s)

# xs_new=xs.fillna(0).astype(int)

# my_model = RandomForestClassifier(random_state=42)

# rf_split = KFold(n_splits=4, shuffle=True, random_state=42)
# fbeta_scorer_s = make_scorer(fbeta_score, beta=0.5)

# x1_s = np.array(xs_new)
# y1_s= np.array(ys)

# for train_s, test_s in rf_split.split(x1_s):
#     x1_train_s, x1_test_s = x1_s[train_s], x1_s[test_s]
#     y1_train_s, y1_test_s = y1_s[train_s], y1_s[test_s]
#     fit = my_model.fit(x1_s[train_s], y1_s[train_s])
#     y1_s_pred = my_model.predict(x1_test_s)


In [74]:
data_df = pd.read_csv("hotel_bookings.csv")
pd.set_option("display.float_format", lambda x: "%.5f" % x)
# pd.options.display.max_rows = 200000
data_df = data_df.drop(columns=["reservation_status_date", "hotel", "reservation_status",
    "arrival_date_week_number", "reserved_room_type", "company"])

data_df["total_guests"] = data_df["adults"] + data_df["children"] + data_df["babies"]
data_df["price_per_guest"] = data_df["adr"]/data_df["total_guests"]

filter = (data_df.children == 0) & (data_df.adults == 0) & (data_df.babies == 0)
data_df[filter]
data_df = data_df[~filter]
data_df=data_df.replace(" ","_")
data_df = data_df.reindex(data_df.columns.tolist(), axis = 1)

nan_replacements = {"children": 0.0, "agent": 0, "country": "unknown", "total_guests":0, "price_per_guest":0}
data_df = data_df.fillna(nan_replacements)

data_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN
data_df.dropna(inplace=True)

data_df.drop(14969)

data_df.head()


,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,total_guests,price_per_guest
0,0,342,2015,July,1,0,0,2,0.00000,0,...,3,No Deposit,0.00000,0,Transient,0.00000,0,0,2.00000,0.00000
1,0,737,2015,July,1,0,0,2,0.00000,0,...,4,No Deposit,0.00000,0,Transient,0.00000,0,0,2.00000,0.00000
2,0,7,2015,July,1,0,1,1,0.00000,0,...,0,No Deposit,0.00000,0,Transient,75.00000,0,0,1.00000,75.00000
3,0,13,2015,July,1,0,1,1,0.00000,0,...,0,No Deposit,304.00000,0,Transient,75.00000,0,0,1.00000,75.00000
4,0,14,2015,July,1,0,2,2,0.00000,0,...,0,No Deposit,240.00000,0,Transient,98.00000,0,1,2.00000,49.00000


In [3]:
features = data_df.drop(columns=["is_canceled","total_guests"], axis=1).columns
xs = data_df[features]
ys = data_df["is_canceled"]
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()
le5 = LabelEncoder()
le6 = LabelEncoder()
le7 = LabelEncoder()
le8 = LabelEncoder()

le1.fit(xs['arrival_date_month'])
tmp_s = le1.transform(xs['arrival_date_month']).reshape((-1,1))
xs['arrival_date_month'] = pd.DataFrame(tmp_s)

le2.fit(xs['meal'])
tmp_s = le2.transform(xs['meal']).reshape((-1,1))
xs['meal'] = pd.DataFrame(tmp_s)

le3.fit(xs['market_segment'])
tmp_s = le3.transform(xs['market_segment']).reshape((-1,1))
xs['market_segment'] = pd.DataFrame(tmp_s)

le4.fit(xs['country'])
tmp_s = le4.transform(xs['country']).reshape((-1,1))
xs['country'] = pd.DataFrame(tmp_s)

le5.fit(xs['distribution_channel'])
tmp_s = le5.transform(xs['distribution_channel']).reshape((-1,1))
xs['distribution_channel'] = pd.DataFrame(tmp_s)

le6.fit(xs['assigned_room_type'])
tmp_s = le6.transform(xs['assigned_room_type']).reshape((-1,1))
xs['assigned_room_type'] = pd.DataFrame(tmp_s)

le7.fit(xs['deposit_type'])
tmp_s = le7.transform(xs['deposit_type']).reshape((-1,1))
xs['deposit_type'] = pd.DataFrame(tmp_s)

le8.fit(xs['customer_type'])
tmp_s = le8.transform(xs['customer_type']).reshape((-1,1))
xs['customer_type'] = pd.DataFrame(tmp_s)

xs_new=xs.fillna(0).astype(int)

C:\Users\iwona\AppData\Local\Temp/ipykernel_22988/3243662368.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xs['arrival_date_month'] = pd.DataFrame(tmp_s)
C:\Users\iwona\AppData\Local\Temp/ipykernel_22988/3243662368.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xs['meal'] = pd.DataFrame(tmp_s)
C:\Users\iwona\AppData\Local\Temp/ipykernel_22988/3243662368.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [5]:
my_model = RandomForestClassifier(random_state=42)

rf_split = KFold(n_splits=4, shuffle=True, random_state=42)
fbeta_scorer_s = make_scorer(fbeta_score, beta=0.5)

In [6]:
warnings.filterwarnings("ignore")
x1_s = np.array(xs_new)
y1_s= np.array(ys)

for train_s, test_s in rf_split.split(x1_s):
    x1_train_s, x1_test_s = x1_s[train_s], x1_s[test_s]
    y1_train_s, y1_test_s = y1_s[train_s], y1_s[test_s]
    fit = my_model.fit(x1_s[train_s], y1_s[train_s])
    y1_s_pred = my_model.predict(x1_test_s)

In [7]:
zb_test = pd.DataFrame(x1_test_s, columns=features)

In [8]:
zb_test['arrival_date_month'] = le1.inverse_transform(zb_test['arrival_date_month'])
zb_test['meal'] = le2.inverse_transform(zb_test['meal'])
zb_test['market_segment'] = le3.inverse_transform(zb_test['market_segment'])
zb_test['country'] = le4.inverse_transform(zb_test['country'])
zb_test['distribution_channel'] = le5.inverse_transform(zb_test['distribution_channel'])
zb_test['assigned_room_type'] = le6.inverse_transform(zb_test['assigned_room_type'])
zb_test['deposit_type'] = le7.inverse_transform(zb_test['deposit_type'])
zb_test['customer_type'] = le8.inverse_transform(zb_test['customer_type'])

In [9]:
zb_test

,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,...,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,price_per_guest
0,7,2015,July,1,0,1,1,0,0,BB,...,C,0,No Deposit,0,0,Transient,75,0,0,75
1,14,2015,July,1,0,2,2,0,0,BB,...,A,0,No Deposit,240,0,Transient,98,0,1,49
2,37,2015,July,1,0,4,2,0,0,BB,...,E,0,No Deposit,241,0,Transient,94,0,0,47
3,68,2015,July,1,0,4,2,0,0,BB,...,E,0,No Deposit,240,0,Transient,97,0,3,48
4,37,2015,July,1,0,4,2,0,0,BB,...,E,0,No Deposit,8,0,Contract,97,0,0,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29797,207,2017,April,25,3,7,2,0,0,BB,...,A,0,No Deposit,14,0,Contract,96,0,2,48
29798,175,2017,April,31,1,3,1,0,0,BB,...,A,0,No Deposit,42,0,Contract,82,0,1,82
29799,185,2017,April,30,1,4,2,0,0,BB,...,A,0,No Deposit,9,0,Contract,99,0,1,49
29800,164,2017,April,31,2,4,2,0,0,BB,...,A,0,No Deposit,42,0,Contract,87,0,0,43


In [10]:
sym = {"wartości rezczywiste": y1_test_s,
        "wartości przewidziane": y1_s_pred}

sym_df = pd.DataFrame(sym)
sym_df

,wartości rezczywiste,wartości przewidziane
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
29797,0,0
29798,0,0
29799,0,0
29800,0,0


In [11]:
sym_df = sym_df.join(zb_test)


In [12]:
sym_df

,wartości rezczywiste,wartości przewidziane,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,price_per_guest
0,0,0,7,2015,July,1,0,1,1,0,...,C,0,No Deposit,0,0,Transient,75,0,0,75
1,0,0,14,2015,July,1,0,2,2,0,...,A,0,No Deposit,240,0,Transient,98,0,1,49
2,0,0,37,2015,July,1,0,4,2,0,...,E,0,No Deposit,241,0,Transient,94,0,0,47
3,0,0,68,2015,July,1,0,4,2,0,...,E,0,No Deposit,240,0,Transient,97,0,3,48
4,0,0,37,2015,July,1,0,4,2,0,...,E,0,No Deposit,8,0,Contract,97,0,0,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29797,0,0,207,2017,April,25,3,7,2,0,...,A,0,No Deposit,14,0,Contract,96,0,2,48
29798,0,0,175,2017,April,31,1,3,1,0,...,A,0,No Deposit,42,0,Contract,82,0,1,82
29799,0,0,185,2017,April,30,1,4,2,0,...,A,0,No Deposit,9,0,Contract,99,0,1,49
29800,0,0,164,2017,April,31,2,4,2,0,...,A,0,No Deposit,42,0,Contract,87,0,0,43


In [13]:
print(sym_df["wartości rezczywiste"].value_counts(), sym_df["wartości przewidziane"].value_counts())

0    18868
1    10934
Name: wartości rezczywiste, dtype: int64 0    20359
1     9443
Name: wartości przewidziane, dtype: int64


In [ ]:
print(confusion_matrix(y1_test_s, y1_s_pred))

In [14]:
def sort_month(df, column_name):
    return sd.Sort_Dataframeby_Month(df, column_name)

In [15]:
# srednia cena za dany miesiąc za noc od osoby
cena_za_miesiac = data_df.groupby(['arrival_date_month'])['adr'].mean().reset_index()
cena_za_miesiac.columns = ['month', 'price']
cena_za_miesiac = sort_month(cena_za_miesiac, 'month')
cena_za_miesiac

,month,price
0,January,70.45163
1,February,73.71705
2,March,80.88428
3,April,100.46815
4,May,108.77140
5,June,116.76149
6,July,126.94470
7,August,140.25530
8,September,105.12729
9,October,88.01140


In [16]:
liczba_gosci = sym_df['arrival_date_month'].value_counts().reset_index()
liczba_gosci.columns=['month','no of guests']
liczba_gosci = sort_month(liczba_gosci,'month')
liczba_gosci

,month,no of guests
0,January,1501
1,February,1981
2,March,2459
3,April,2837
4,May,2981
5,June,2667
6,July,3060
7,August,3519
8,September,2678
9,October,2746


In [17]:
final = cena_za_miesiac.merge(liczba_gosci,on='month')
final.columns=['month','cena za miesiąc', 'liczba zajetych pokoi']
final["cena_calkowita"] = final['cena za miesiąc'] * final['liczba zajetych pokoi']
final["status"]  = "overall"
final

,month,cena za miesiąc,liczba zajetych pokoi,cena_calkowita,status
0,January,70.45163,1501,105747.89885,overall
1,February,73.71705,1981,146033.48427,overall
2,March,80.88428,2459,198894.45030,overall
3,April,100.46815,2837,285028.12985,overall
4,May,108.77140,2981,324247.54289,overall
5,June,116.76149,2667,311402.89036,overall
6,July,126.94470,3060,388450.79488,overall
7,August,140.25530,3519,493558.41002,overall
8,September,105.12729,2678,281530.88134,overall
9,October,88.01140,2746,241679.30790,overall


In [18]:
px.bar(final, x = 'month', y = 'cena_calkowita')

In [19]:
odwolane = sym_df[sym_df['wartości przewidziane'] == 1]
nieodwolane = sym_df[sym_df['wartości przewidziane'] == 0]

In [20]:
liczba_gosci = nieodwolane['arrival_date_month'].value_counts().reset_index()
liczba_gosci.columns=['month','liczba zajetych pokoi']
liczba_gosci = sort_month(liczba_gosci,'month')
liczba_gosci

,month,liczba zajetych pokoi
0,January,1147
1,February,1485
2,March,1810
3,April,1817
4,May,1918
5,June,1695
6,July,2006
7,August,2355
8,September,1760
9,October,1894


In [21]:
final_nd = cena_za_miesiac.merge(liczba_gosci,on='month')
final_nd.columns=['month','cena za miesiąc', 'liczba zajetych pokoi']
final_nd["cena_calkowita"] = final_nd['cena za miesiąc'] * final_nd['liczba zajetych pokoi']
final_nd["status"]  = "non calceled"
final_nd

,month,cena za miesiąc,liczba zajetych pokoi,cena_calkowita,status
0,January,70.45163,1147,80808.02131,non calceled
1,February,73.71705,1485,109469.82541,non calceled
2,March,80.88428,1810,146400.55105,non calceled
3,April,100.46815,1817,182550.62105,non calceled
4,May,108.77140,1918,208623.54487,non calceled
5,June,116.76149,1695,197910.72335,non calceled
6,July,126.94470,2006,254651.07664,non calceled
7,August,140.25530,2355,330301.23774,non calceled
8,September,105.12729,1760,185024.02956,non calceled
9,October,88.01140,1894,166693.59401,non calceled


In [22]:
sym_df['deposit_type'].value_counts()

No Deposit    26124
Non Refund     3640
Refundable       38
Name: deposit_type, dtype: int64

In [23]:
odwolane_z_depozytem = odwolane[odwolane['deposit_type'] == 'Non Refund']
odwolane_z_depozytem


,wartości rezczywiste,wartości przewidziane,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,price_per_guest
559,1,1,87,2015,October,3,0,1,2,0,...,A,0,Non Refund,96,0,Transient,36,0,0,18
569,1,1,170,2015,October,4,4,8,2,0,...,A,0,Non Refund,15,0,Transient,44,0,0,22
591,1,1,78,2015,October,9,1,2,2,0,...,A,0,Non Refund,134,0,Transient,92,0,0,46
592,1,1,78,2015,October,9,1,2,2,0,...,A,0,Non Refund,134,0,Transient,92,0,0,46
593,1,1,78,2015,October,9,1,2,2,0,...,A,0,Non Refund,134,0,Transient,92,0,0,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20543,1,1,68,2016,May,17,0,2,2,0,...,A,0,Non Refund,37,0,Transient,75,0,0,37
20544,1,1,68,2016,May,17,0,2,2,0,...,A,0,Non Refund,37,0,Transient,75,0,0,37
20545,1,1,68,2016,May,17,0,2,2,0,...,A,0,Non Refund,37,0,Transient,75,0,0,37
20546,1,1,68,2016,May,17,0,2,2,0,...,A,0,Non Refund,37,0,Transient,75,0,0,37


In [24]:
#zaliczka wynosi 30%
cena_za_miesiac['price'] = cena_za_miesiac['price']*0.3
cena_za_miesiac

,month,price
0,January,21.13549
1,February,22.11512
2,March,24.26528
3,April,30.14044
4,May,32.63142
5,June,35.02845
6,July,38.08341
7,August,42.07659
8,September,31.53819
9,October,26.40342


In [25]:
liczba_gosci = odwolane_z_depozytem['arrival_date_month'].value_counts().reset_index()
liczba_gosci.columns=['month','liczba zajetych pokoi']
liczba_gosci = sort_month(liczba_gosci,'month')
liczba_gosci

,month,liczba zajetych pokoi
0,January,120
1,February,181
2,March,238
3,April,407
4,May,415
5,June,412
6,July,158
7,August,216
8,September,402
9,October,360


In [26]:
final_od = cena_za_miesiac.merge(liczba_gosci,on='month')
final_od.columns=['month','cena za miesiąc', 'liczba zajetych pokoi']
final_od["cena_calkowita"] = final_od['cena za miesiąc'] * final_od['liczba zajetych pokoi']
final_od["status"]  = "non calceled"
final_od

,month,cena za miesiąc,liczba zajetych pokoi,cena_calkowita,status
0,January,21.13549,120,2536.25873,non calceled
1,February,22.11512,181,4002.83604,non calceled
2,March,24.26528,238,5775.13776,non calceled
3,April,30.14044,407,12267.16061,non calceled
4,May,32.63142,415,13542.03928,non calceled
5,June,35.02845,412,14431.72000,non calceled
6,July,38.08341,158,6017.17898,non calceled
7,August,42.07659,216,9088.54361,non calceled
8,September,31.53819,402,12678.35112,non calceled
9,October,26.40342,360,9505.23134,non calceled


In [27]:
z_zaliczka = final_od[['month','cena_calkowita']]

z_zaliczka = z_zaliczka.merge(final_nd[['month','cena_calkowita']], on='month')
z_zaliczka['cena_calkowita'] = z_zaliczka['cena_calkowita_x'] + z_zaliczka['cena_calkowita_y']
z_zaliczka_new = z_zaliczka[['month','cena_calkowita']]
z_zaliczka_new["status"]  = "non calceled + non refund deposit"
z_zaliczka_new



,month,cena_calkowita,status
0,January,83344.28004,non calceled + non refund deposit
1,February,113472.66145,non calceled + non refund deposit
2,March,152175.68881,non calceled + non refund deposit
3,April,194817.78167,non calceled + non refund deposit
4,May,222165.58415,non calceled + non refund deposit
5,June,212342.44335,non calceled + non refund deposit
6,July,260668.25562,non calceled + non refund deposit
7,August,339389.78135,non calceled + non refund deposit
8,September,197702.38068,non calceled + non refund deposit
9,October,176198.82535,non calceled + non refund deposit


In [28]:
porowniae_1 = pd.concat([final[['month','cena_calkowita','status']], (z_zaliczka_new[['month','cena_calkowita','status']])])
porowniae_1

,month,cena_calkowita,status
0,January,105747.89885,overall
1,February,146033.48427,overall
2,March,198894.45030,overall
3,April,285028.12985,overall
4,May,324247.54289,overall
5,June,311402.89036,overall
6,July,388450.79488,overall
7,August,493558.41002,overall
8,September,281530.88134,overall
9,October,241679.30790,overall


In [29]:
px.bar(porowniae_1, x = 'month', y = ['cena_calkowita'], color='status', barmode='group')

In [30]:
#załózmy ze do wszystkich pozostalych dodajmemy zaliczke
sym_1 = odwolane
sym_1

,wartości rezczywiste,wartości przewidziane,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,price_per_guest
9,0,1,77,2015,July,1,2,5,2,0,...,A,0,No Deposit,240,0,Transient,94,0,0,47
12,0,1,36,2015,July,2,1,3,3,0,...,D,0,No Deposit,241,0,Transient,108,0,1,36
14,1,1,47,2015,July,2,2,5,2,2,...,G,0,No Deposit,240,0,Transient,153,0,0,38
27,1,1,51,2015,July,3,0,2,3,0,...,A,0,No Deposit,242,0,Transient,110,0,0,36
28,1,1,48,2015,July,3,1,2,2,0,...,E,0,No Deposit,240,0,Transient,123,0,0,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29650,0,1,230,2017,August,21,1,4,2,0,...,A,0,No Deposit,9,0,Transient,117,0,2,58
29705,0,1,48,2017,August,25,1,2,2,0,...,A,0,No Deposit,9,0,Transient,121,0,1,60
29727,0,1,269,2017,August,22,2,5,2,0,...,A,0,No Deposit,9,0,Transient,99,0,2,49
29735,0,1,267,2017,August,25,2,2,3,0,...,D,0,No Deposit,9,0,Transient,166,0,2,55


In [31]:
liczba_gosci = sym_1['arrival_date_month'].value_counts().reset_index()
liczba_gosci.columns=['month','liczba zajetych pokoi']
liczba_gosci = sort_month(liczba_gosci,'month')
liczba_gosci

,month,liczba zajetych pokoi
0,January,354
1,February,496
2,March,649
3,April,1020
4,May,1063
5,June,972
6,July,1054
7,August,1164
8,September,918
9,October,852


In [32]:
final_sym1 = cena_za_miesiac.merge(liczba_gosci,on='month')
final_sym1.columns=['month','cena za miesiąc', 'liczba zajetych pokoi']
final_sym1["cena_calkowita"] = final_sym1['cena za miesiąc'] * final_sym1['liczba zajetych pokoi']
final_sym1["status"]  = "extra depisit"
final_sym1

,month,cena za miesiąc,liczba zajetych pokoi,cena_calkowita,status
0,January,21.13549,354,7481.96326,extra depisit
1,February,22.11512,496,10969.09766,extra depisit
2,March,24.26528,649,15748.16977,extra depisit
3,April,30.14044,1020,30743.25264,extra depisit
4,May,32.63142,1063,34687.19941,extra depisit
5,June,35.02845,972,34047.65010,extra depisit
6,July,38.08341,1054,40139.91547,extra depisit
7,August,42.07659,1164,48977.15168,extra depisit
8,September,31.53819,918,28952.05553,extra depisit
9,October,26.40342,852,22495.71417,extra depisit


In [33]:
f1 = final_sym1[['month','cena_calkowita']]
f1 = f1.merge(final_nd[['month','cena_calkowita']], on='month')
f1['cena_calkowita'] = f1['cena_calkowita_x'] + f1['cena_calkowita_y']
f1_new = f1[['month','cena_calkowita']]
f1_new["status"]  = "non calceled + non refund deposit + extra deposit"
f1_new

,month,cena_calkowita,status
0,January,88289.98457,non calceled + non refund deposit + extra deposit
1,February,120438.92307,non calceled + non refund deposit + extra deposit
2,March,162148.72083,non calceled + non refund deposit + extra deposit
3,April,213293.87369,non calceled + non refund deposit + extra deposit
4,May,243310.74428,non calceled + non refund deposit + extra deposit
5,June,231958.37345,non calceled + non refund deposit + extra deposit
6,July,294790.99211,non calceled + non refund deposit + extra deposit
7,August,379278.38942,non calceled + non refund deposit + extra deposit
8,September,213976.08510,non calceled + non refund deposit + extra deposit
9,October,189189.30818,non calceled + non refund deposit + extra deposit


In [34]:
# porownanie_2 = pd.concat([porowniae_1, (final_sym1[['month','cena_calkowita','status']])])
porownanie_2 = pd.concat([porowniae_1[['month','cena_calkowita','status']], f1_new[['month','cena_calkowita','status']]])
porownanie_2

,month,cena_calkowita,status
0,January,105747.89885,overall
1,February,146033.48427,overall
2,March,198894.45030,overall
3,April,285028.12985,overall
4,May,324247.54289,overall
5,June,311402.89036,overall
6,July,388450.79488,overall
7,August,493558.41002,overall
8,September,281530.88134,overall
9,October,241679.30790,overall


In [35]:
px.bar(porownanie_2, x = 'month', y = ['cena_calkowita'], color='status', barmode='group')

In [36]:
odwolane_dlugi_lead_lime = odwolane[odwolane['lead_time'] >= 14]
odwolane_dlugi_lead_lime

,wartości rezczywiste,wartości przewidziane,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,price_per_guest
9,0,1,77,2015,July,1,2,5,2,0,...,A,0,No Deposit,240,0,Transient,94,0,0,47
12,0,1,36,2015,July,2,1,3,3,0,...,D,0,No Deposit,241,0,Transient,108,0,1,36
14,1,1,47,2015,July,2,2,5,2,2,...,G,0,No Deposit,240,0,Transient,153,0,0,38
27,1,1,51,2015,July,3,0,2,3,0,...,A,0,No Deposit,242,0,Transient,110,0,0,36
28,1,1,48,2015,July,3,1,2,2,0,...,E,0,No Deposit,240,0,Transient,123,0,0,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29650,0,1,230,2017,August,21,1,4,2,0,...,A,0,No Deposit,9,0,Transient,117,0,2,58
29705,0,1,48,2017,August,25,1,2,2,0,...,A,0,No Deposit,9,0,Transient,121,0,1,60
29727,0,1,269,2017,August,22,2,5,2,0,...,A,0,No Deposit,9,0,Transient,99,0,2,49
29735,0,1,267,2017,August,25,2,2,3,0,...,D,0,No Deposit,9,0,Transient,166,0,2,55


In [60]:
# znizca 25%
cena_za_miesiac = data_df.groupby(['arrival_date_month'])['adr'].mean().reset_index()
cena_za_miesiac.columns = ['month', 'price']
cena_za_miesiac = sort_month(cena_za_miesiac, 'month')
cena_za_miesiac['price'] = cena_za_miesiac['price']*0.75
cena_za_miesiac

,month,price
0,January,52.83872
1,February,55.28779
2,March,60.66321
3,April,75.35111
4,May,81.57855
5,June,87.57112
6,July,95.20853
7,August,105.19148
8,September,78.84547
9,October,66.00855


In [61]:
liczba_gosci = odwolane_dlugi_lead_lime['arrival_date_month'].value_counts().reset_index()
liczba_gosci.columns=['month','liczba zajetych pokoi']
liczba_gosci = sort_month(liczba_gosci,'month')
liczba_gosci['liczba zajetych pokoi'] = liczba_gosci['liczba zajetych pokoi']*0.50
liczba_gosci

,month,liczba zajetych pokoi
0,January,162.50000
1,February,221.50000
2,March,307.50000
3,April,499.00000
4,May,507.50000
5,June,476.00000
6,July,517.50000
7,August,566.00000
8,September,441.50000
9,October,416.50000


In [62]:
final_sym2 = cena_za_miesiac.merge(liczba_gosci,on='month')
final_sym2.columns=['month','cena za miesiąc', 'liczba zajetych pokoi']
final_sym2["cena_calkowita"] = final_sym2['cena za miesiąc'] * final_sym2['liczba zajetych pokoi']
final_sym2["status"]  = "25% discount"
final_sym2

,month,cena za miesiąc,liczba zajetych pokoi,cena_calkowita,status
0,January,52.83872,162.50000,8586.29259,25% discount
1,February,55.28779,221.50000,12246.24562,25% discount
2,March,60.66321,307.50000,18653.93762,25% discount
3,April,75.35111,499.00000,37600.20359,25% discount
4,May,81.57855,507.50000,41401.11406,25% discount
5,June,87.57112,476.00000,41683.85147,25% discount
6,July,95.20853,517.50000,49270.41332,25% discount
7,August,105.19148,566.00000,59538.37597,25% discount
8,September,78.84547,441.50000,34810.27374,25% discount
9,October,66.00855,416.50000,27492.56147,25% discount


In [63]:
f2 = final_sym2[['month','cena_calkowita']]
f2 = f2.merge(f1_new[['month','cena_calkowita']], on='month')
f2['cena_calkowita'] = f2['cena_calkowita_x'] + f2['cena_calkowita_y']
f2_new = f2[['month','cena_calkowita']]
f2_new["status"]  = "non calceled + non refund deposit + extra deposit + 25% discount"
f2_new

,month,cena_calkowita,status
0,January,96876.27716,non calceled + non refund deposit + extra depo...
1,February,132685.16869,non calceled + non refund deposit + extra depo...
2,March,180802.65845,non calceled + non refund deposit + extra depo...
3,April,250894.07729,non calceled + non refund deposit + extra depo...
4,May,284711.85834,non calceled + non refund deposit + extra depo...
5,June,273642.22492,non calceled + non refund deposit + extra depo...
6,July,344061.40543,non calceled + non refund deposit + extra depo...
7,August,438816.76539,non calceled + non refund deposit + extra depo...
8,September,248786.35884,non calceled + non refund deposit + extra depo...
9,October,216681.86965,non calceled + non refund deposit + extra depo...


In [64]:
porownanie_3 = pd.concat([porownanie_2[['month','cena_calkowita','status']], f2_new[['month','cena_calkowita','status']]])
porownanie_3

,month,cena_calkowita,status
0,January,105747.89885,overall
1,February,146033.48427,overall
2,March,198894.45030,overall
3,April,285028.12985,overall
4,May,324247.54289,overall
5,June,311402.89036,overall
6,July,388450.79488,overall
7,August,493558.41002,overall
8,September,281530.88134,overall
9,October,241679.30790,overall


In [65]:
px.bar(porownanie_3, x = 'month', y = ['cena_calkowita'], color='status', barmode='group')

In [66]:
odwolane_krotki_lead_lime = odwolane[odwolane['lead_time'] < 14]
odwolane_krotki_lead_lime

,wartości rezczywiste,wartości przewidziane,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,price_per_guest
68,1,1,1,2015,July,8,0,1,2,0,...,A,0,No Deposit,0,0,Transient,104,0,1,52
157,1,1,7,2015,July,20,3,8,2,0,...,A,0,No Deposit,196,0,Transient,200,0,0,100
211,1,1,10,2015,August,1,0,1,2,0,...,F,0,No Deposit,0,0,Transient-Party,202,0,0,101
300,1,1,12,2015,August,16,1,0,2,0,...,E,0,No Deposit,110,0,Transient,184,0,0,92
301,1,1,12,2015,August,16,1,0,2,0,...,E,0,No Deposit,110,0,Transient,184,0,0,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27029,0,1,11,2017,March,21,0,3,2,0,...,G,0,No Deposit,9,0,Transient,98,0,0,49
27265,0,1,8,2017,April,5,0,2,2,2,...,D,0,No Deposit,9,0,Transient,224,0,0,56
28425,0,1,10,2017,June,11,2,0,3,0,...,A,0,No Deposit,9,0,Transient,190,0,0,63
29155,0,1,13,2017,July,22,2,2,3,0,...,A,0,No Deposit,9,0,Transient-Party,199,0,1,66


In [72]:
# znizka 50%
cena_za_miesiac = data_df.groupby(['arrival_date_month'])['adr'].mean().reset_index()
cena_za_miesiac.columns = ['month', 'price']
cena_za_miesiac = sort_month(cena_za_miesiac, 'month')
cena_za_miesiac['price'] = cena_za_miesiac['price']*0.50
cena_za_miesiac

# załozmt ze tylko polowa pokoi zostanie zarezerwowana
liczba_gosci = odwolane_krotki_lead_lime['arrival_date_month'].value_counts().reset_index()
liczba_gosci.columns=['month','liczba zajetych pokoi']
liczba_gosci = sort_month(liczba_gosci,'month')
liczba_gosci['liczba zajetych pokoi'] = liczba_gosci['liczba zajetych pokoi']*0.50
liczba_gosci

final_sym3 = cena_za_miesiac.merge(liczba_gosci,on='month')
final_sym3.columns=['month','cena za miesiąc', 'liczba zajetych pokoi']
final_sym3["cena_calkowita"] = final_sym2['cena za miesiąc'] * final_sym3['liczba zajetych pokoi']
final_sym3["status"]  = "50% discount"
final_sym3

f3 = final_sym3[['month','cena_calkowita']]
f3 = f3.merge(f2_new[['month','cena_calkowita']], on='month')
f3['cena_calkowita'] = f3['cena_calkowita_x'] + f3['cena_calkowita_y']
f3_new = f3[['month','cena_calkowita']]
f3_new["status"]  = "non calceled + non refund deposit + extra deposit + 25% discount + 50% discount"
f3_new

porownanie_4 = pd.concat([porownanie_3[['month','cena_calkowita','status']], f3_new[['month','cena_calkowita','status']]])
porownanie_4.to_csv("symulacja_4.csv")



# px.bar(porownanie_4, x = 'month', y = ['cena_calkowita'], color='status', barmode='group')

In [ ]:
fig1 = px.line(final, x = 'month', y = 'cena_calkowita', coloraxis="coloraxis")
fig2 = px.line(z_zaliczka_new, x = 'month', y = 'cena_calkowita', coloraxis="coloraxis")

fig = go.Figure(data = fig1.data + fig2.data)
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.bar(porownanie_2, x = 'month', y = ['cena_calkowita'], color='status'))


fig.add_trace(go.bar(final, x = 'month', y = ['cena_calkowita']))




fig.show()